# Redes Neurais Recorrentes

### Descriçao

---
1. O objetivo deste notebook é construir uma Rede Neural Recorrente.
2. Utilizaremos o conjunto de dados proposto no [gist treino](https://gist.githubusercontent.com/batestin1/b8f7c0a26c9669013ba451c18e381d75/raw/e355693a14880cd8dbe2bee26a52dafdc9aaabec/gistfile1.txt) e [gist test](https://gist.githubusercontent.com/batestin1/f94107b82cc271c203c1d5843b8dded3/raw/f5e653d96b45bb19773230b5090c78f0695f8919/gistfile1.txt)

3. O problema consiste em prever as ações temporais do preço da Google na bolsa de valores
---

### Dicionário


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
Date 	  										  	  |string     | Data da alteração |
Open														  |float    | preço da abertura                        |
High		     										  |float     | preço mais alto no dia	               |
Low | float | preco mais baixo no dia
Close | float | preco de fechamento
Volume | float | Volume total do dia
  

# Instalação dos pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras matplotlib

# Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)



# Instalando as bibliotecas




In [ ]:
import numpy as np #para manipulacao e criacao de matrizes
import matplotlib.pyplot as plt #para visualizacao dos dados
import pandas as pd #para manipulacao de dados

from sklearn.preprocessing import MinMaxScaler # para pre processamento de ml

from keras.models import Sequential # para iniciar nossa rede neural
from keras.layers import Dense #para criar os neuronios
from keras.layers import LSTM #para corrigir nosso vanishing gradient, a solucao LSTM
from keras.layers import Dropout #para reduzir o overfitting

import os # para criacao e manipulacao de pastas
from keras.models import load_model #para salvar modelos do keras

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


# Obtendo o dataset

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/batestin1/b8f7c0a26c9669013ba451c18e381d75/raw/e355693a14880cd8dbe2bee26a52dafdc9aaabec/gistfile1.txt')


#### Conhecendo o dataset

In [ ]:
df.info()

In [ ]:
#vendo o periodo
ano_inicial = df['Date'].min()
ano_final = df['Date'].max()

print(ano_inicial[5:], ano_final[4:])

# Pré Processamento

In [ ]:
# Separando os dados de treino
training_set = df.iloc[:, 1:2].values #aqui estou pegando uma array da coluna 'open' o valor de abertura do mercado





In [ ]:
num = 115
print(f"comparando o training_set: {training_set[num][0]} X coluna open: {df['Open'][num]}")



In [ ]:
sc = MinMaxScaler(feature_range = (0, 1)) #padronizando os dados de treino. O feature_range é um alcance entre 0 e 1
training_set_scaled = sc.fit_transform(training_set) #aplicando o metodo
training_set_scaled

In [ ]:
training_set_scaled.shape

In [ ]:
#Estruturas de dados com 60 intervalos
X_train = []
y_train = []
for i in range(60, len(training_set_scaled)): #este 60 indica o inicio de nosso range, o modelo vai olhar 60 dias para trás para prever o próximo da frente.
    X_train.append(training_set_scaled[i-60:i, 0]) #o indice i-60:i, 0], indica que esta pegando todos do i, 60 intervalos para trás, jogando na nossa lista vazia X_train
    y_train.append(training_set_scaled[i, 0]) #jogando na nossa lista vazia y_train
X_train, y_train = np.array(X_train), np.array(y_train) #criando uma matriz para X e para y

In [ ]:
X_train.shape

In [ ]:
# Ajustar Formato
#transformando em uma matriz em tres dimensao,

parametros = (
    X_train.shape[0], #batch size, o numero de linhas de nossa matriz
    X_train.shape[1], #o numero de passos a serem realizados (tmb o numero de colunas)
    1 #nosso indicador, nosso indicador vai ser o Open
)


X_train = np.reshape(X_train, (parametros))
X_train.shape

In [ ]:
X_train

# Construindo nossa RNR

In [ ]:
#Inicializando a RNA
rnr = Sequential()

In [ ]:

#Criando as camadas
rnr.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1))) #a primeira camada é LSTM, o return_sequence é do LSTM e entende que ele precisa retorna em forma de sequencia. o input_shape só precisa de duas dimensoes, pq a quantidade de observacoes é automatico
rnr.add(Dropout(0.2)) #para reduzir o overfitting, o parametro 0,2 indica o numero de neuronios que ele desativa apos a leitura da primeira.
rnr.add(LSTM(units = 50, return_sequences = True))
rnr.add(Dropout(0.2))
rnr.add(LSTM(units = 50, return_sequences = True))
rnr.add(Dropout(0.2))
rnr.add(LSTM(units = 50))
rnr.add(Dropout(0.2))
rnr.add(Dense(units = 1)) #para conectar todos os neuronios. A camada de saida.

In [ ]:
#compilando

rnr.compile(optimizer = 'adam', loss='mean_squared_error') # como se trata de um problema de regressao, a metrica q funcao de erro tem que usar é o mean_squared_error


In [ ]:
#treinando o modelo

rnr.fit(X_train, y_train, epochs = 100, batch_size = 32)

# Testando

In [ ]:
dados_de_teste = pd.read_csv('https://gist.githubusercontent.com/batestin1/f94107b82cc271c203c1d5843b8dded3/raw/f5e653d96b45bb19773230b5090c78f0695f8919/gistfile1.txt')

dados_de_teste

In [ ]:
abertura_real = dados_de_teste.iloc[:,1:2].values #pegando os dados da coluna Open
abertura_real

In [ ]:
dataset_total = pd.concat((df['Open'], dados_de_teste['Open']), axis = 0) #concatenando os dois valores, o teste e o treino do original
dataset_total

In [ ]:
inputs = dataset_total[len(dataset_total) - len(dados_de_teste) - 60:].values #pegando o 60 dias antes do ultimo dia de janeiro de 2017 até o ultimo dia de 2016

In [ ]:
inputs.shape

In [ ]:
#organizando ela para o formato de matriz que precisamos para testar

inputs = inputs.reshape(-1,1)
inputs.shape

In [ ]:
#padronizacao dos numeros
inputs = sc.transform(inputs)
inputs

In [ ]:
#utilizando a mesma preparacao que fizemos para o treino, agora para o teste.
X_test = []
for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
#obtendo a metrica da previsao

print(f"previsao oficial: {rnr.predict(X_test)}")
print(f"previsao revertida: {sc.inverse_transform(rnr.predict(X_test))}")

variavel_final = sc.inverse_transform(rnr.predict(X_test))


In [ ]:
abertura_real[9][0]

In [ ]:
print(f"abertura oficial: {abertura_real[7][0]}")
print(f"previsao obtida {round(sc.inverse_transform(rnr.predict(X_test))[7][0])}")

#variavel_final = sc.inverse_transform(rnr.predict(X_test))

In [ ]:
dados_de_teste[dados_de_teste['Open'] == 807.14]

In [ ]:
#calculando a previsao do modelo
valor_final = round(sc.inverse_transform(rnr.predict(X_test))[7][0],2)
valor_inicial = abertura_real[7][0]
diferenca_percentual = ((valor_inicial - valor_final ) / valor_inicial) * 100
print(f"Diferença percentual: {diferenca_percentual:.2f}%")

# Salvando o modelo

In [ ]:
folder = 'rnr/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
    os.makedirs(folder)

# Salva o modelo no diretório especificado
rnr.save(os.path.join(folder, 'rede_neural_recorrente.h5'))

# Importando o modelo

In [ ]:
model = load_model(os.path.join(folder, 'rede_neural_recorrente.h5'))

# Visualizando o Resultado

In [ ]:
plt.plot(abertura_real, color = 'red', label = 'Dados Reais de Ações do Google')
plt.plot(variavel_final, color = 'blue', label = 'Dados Previstos de Ações do Google')
plt.title('Previsão de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações do Google')
plt.legend()
plt.show()